In [3]:
#Model card: https://huggingface.co/databricks/dolly-v2-3b

In [6]:
%pip install "accelerate>=0.16.0,<1" "transformers[torch]>=4.28.1,<5" "torch>=1.13.1,<2"


Defaulting to user installation because normal site-packages is not writeable
  Using cached accelerate-0.23.0-py3-none-any.whl (258 kB)
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement torch<2,>=1.13.1 (from versions: 2.0.0, 2.0.1, 2.1.0)
ERROR: No matching distribution found for torch<2,>=1.13.1

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import torch
from transformers import pipeline
!pip install import-ipynb
import import_ipynb

generate_text = pipeline(model="databricks/dolly-v2-3b", torch_dtype=torch.bfloat16, trust_remote_code=True, device_map="auto")


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.3 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
res = generate_text("Provide a yes or no answer to the following question: Can you eat outside at taco bell?")
print(res[0]["generated_text"])
res = generate_text("Provide a yes or no answer to the following question: can you drink alcoholic drink in populace in kingdom of denmark?")
print(res[0]["generated_text"])

Yes, you can eat outside at Taco Bell if the temperature is below 100 degrees fahrenheit.
Yes, can you drink alcoholic drink in populace in kingdom of denmark?

The question itself is relative. It depends on your view point. If we are taking into account the people of kingdom of Denmark, yes, you can drink alcoholic drink. As they are of the blooded people, they have no distinction between brewery or distillery.


In [9]:


from ipynb.fs.full.Synonym_Generation import *

def response(sent):
    out = generate_text(sent)
    return out

sentence_answers = []
def sentence_input(verified_ca):
    sentences = []
    for row in verified_ca:
        checked = 'Provide a yes or no answer to the following question: '
        checked += ' '.join(row)
        sentences.append(checked)
    res = list(map(response, sentences))
    total_possibilities = len(sentences)
    results = []
    for answer in res:
        answer = answer[0]["generated_text"]
        if answer.lower().startswith('yes'):
            results.append(1)
        elif answer.lower().startswith('no'):
            results.append(0)
        else: 
            results.append(0)      
    return res, results, total_possibilities

def LA_file(results, verified_array, out_file_name):
    with open(out_file_name, 'w') as out_file:
        out_file.write('v2.0' + '\n')
        out_file.write(str(len(results)) + '\n')
        for sent in verified_array:
            for word in sent:
                out_file.write(str(word) + '\t')
            out_file.write('\n')

def response_file(results, out_file_name):
    with open(out_file_name, 'w') as out_file:
        out_file.write(str(len(results)) + '\n')
        out_file.write('Throughput' + '\n')
        for result in results:
            out_file.write(str(result) + '\t')
            out_file.write('\n')

def factor_file(original_sentence: str, synonym_nest, out_file_name):
    with open(out_file_name, 'w') as out_file:
        out_file.write(str(len(original_sentence)))
        original_sentence = original_sentence.split()
        for word, syn_list in zip(original_sentence, synonym_nest):
            out_file.write(word + '\t')
            out_file.write('\t'.join(syn_list))
            out_file.write('\n')
            # out_file.write(word + '\t')
            # for syn_list in synonym_nest:
            #     out_file.write(str(len(syn_list)) + '\t')
            #     out_file.write('0' + '\t')
            #     for syn in syn_list:
            #         out_file.write(syn + '\t')
            #     out_file.write('\n')
#specific synonym from original sentence, number of synonyms, ? on this part, and then list the synonyms with tab separation

def acc_checker(res, total_possibilities):
    number_correct = 0
    number_incorrect = 0
    uncertain = 0
    for answer in res:
        answer = answer[0]["generated_text"]
        if answer.lower().startswith('yes'):
            number_correct += 1
        elif answer.lower().startswith('no'):
            number_incorrect += 1
        else: 
            uncertain += 1
        accuracy = number_correct / total_possibilities
    return accuracy

responses, results, total_possibilities = sentence_input(tt)
LA_file(results, tt, 'LA_file')
response_file(results, 'response_file')
factor_file(syn_sentence, sent_array, 'factor_file')

In [1]:
!pip install datasets
from datasets import load_dataset

dataset = load_dataset("boolq")
#dataset = load_dataset("metaeval/boolq-natural-perturbations")

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.3 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from ipynb.fs.full.Synonym_Generation import *
import csv


t = 2
N = 10
d = 1
lamb = 2

accumulator = 0
while accumulator != 3:
    for qs in dataset['validation']:
        qs = dataset['validation'][accumulator]['question']
        test_list = []
        qs_array = repeat(syn_generator(qs), N, t, d, lamb)
        #print(qs_array)
        test_list.append(sentence_input(qs_array))
        accumulator += 1

print(test_list)

with open('synonym_file.csv', 'w') as s_file:
    wr = csv.writer(s_file)
    wr.writerows(test_list)

for result in test_list:
    print(result)
    valid_sum = 0
    valid_sum += int(result)
valid_avg = valid_sum / len(test_list)

print(valid_avg)

NameError: name 'dataset' is not defined

In [ ]:
#addition lines of code, just to be saved in case they are needed again for reference

  # for row in verified_ca:
    #     row += '? The boolean answer to this question is yes.'
    #     res = generate_text(row)
    #     answer = res[0]["generated_text"]
    #     sentence_answers.append(answer)




# baseline_sentence = 'can you drink alcoholic drink in populace in kingdom of denmark'
# res = generate_text(baseline_sentence)

# for row in ca_array:
#   sentence = ''
#   sentence = ' '.join(row)
#   res = generate_text(sentence)

#largely looking for a boolean output


# res = generate_text("Explain to me the difference between nuclear fission and fusion.")
# print(res[0]["generated_text"])

In [8]:
# %load Synonym_Generation.ipynb
{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": 9,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "['ace', 'adept', 'champion', 'sensation', 'maven', 'mavin', 'virtuoso', 'genius', 'hotshot', 'star', 'superstar', 'whiz', 'whizz', 'wizard', 'wiz']\n",
      "['whiz']\n",
      "['whizz', 'whiz', 'whirr', 'whir', 'birr', 'purr']\n"
     ]
    }
   ],
   "source": [
    "from itertools import chain\n",
    "from nltk.corpus import wordnet\n",
    "# for synset in wordnet.synsets('whiz'): \n",
    "#     print(synset.lemma_names())"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 8,
   "metadata": {},
   "outputs": [
    {
     "name": "stderr",
     "output_type": "stream",
     "text": [
      "[nltk_data] Downloading package wordnet to\n",
      "[nltk_data]     C:\\Users\\william.gregory\\AppData\\Roaming\\nltk_data...\n"
     ]
    },
    {
     "data": {
      "text/plain": [
       "True"
      ]
     },
     "execution_count": 8,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "import nltk\n",
    "nltk.download('wordnet')"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 13,
   "metadata": {},
   "outputs": [
    {
     "ename": "UnsupportedOperation",
     "evalue": "not readable",
     "output_type": "error",
     "traceback": [
      "\u001b[1;31m---------------------------------------------------------------------------\u001b[0m",
      "\u001b[1;31mUnsupportedOperation\u001b[0m                      Traceback (most recent call last)",
      "\u001b[1;32mUntitled-1.ipynb Cell 3\u001b[0m line \u001b[0;36m1\n\u001b[0;32m     <a href='vscode-notebook-cell:Untitled-1.ipynb?jupyter-notebook#W2sdW50aXRsZWQ%3D?line=9'>10</a>\u001b[0m string_similar_words \u001b[39m=\u001b[39m \u001b[39mstr\u001b[39m(similar_words)\n\u001b[0;32m     <a href='vscode-notebook-cell:Untitled-1.ipynb?jupyter-notebook#W2sdW50aXRsZWQ%3D?line=10'>11</a>\u001b[0m syn_file\u001b[39m.\u001b[39mwrite(string_similar_words)\n\u001b[1;32m---> <a href='vscode-notebook-cell:Untitled-1.ipynb?jupyter-notebook#W2sdW50aXRsZWQ%3D?line=11'>12</a>\u001b[0m \u001b[39mfor\u001b[39;49;00m added_word \u001b[39min\u001b[39;49;00m syn_file:\n\u001b[0;32m     <a href='vscode-notebook-cell:Untitled-1.ipynb?jupyter-notebook#W2sdW50aXRsZWQ%3D?line=12'>13</a>\u001b[0m     added_word \u001b[39m=\u001b[39;49m added_word\u001b[39m.\u001b[39;49mstrip()\n\u001b[0;32m     <a href='vscode-notebook-cell:Untitled-1.ipynb?jupyter-notebook#W2sdW50aXRsZWQ%3D?line=13'>14</a>\u001b[0m     w1 \u001b[39m=\u001b[39;49m wordnet\u001b[39m.\u001b[39;49msynset(word)\n",
      "\u001b[1;31mUnsupportedOperation\u001b[0m: not readable"
     ]
    }
   ],
   "source": [
    "# synonym_sentence = 'can you drink alcoholic drink in populace in kingdom of denmark?'\n",
    "# highest_sim_level = 0\n",
    "# most_acc_word = ''\n",
    "# split_sentence = synonym_sentence.strip()\n",
    "# for word in split_sentence:\n",
    "#     for synset in wordnet.synsets(word):\n",
    "#         synonym_list = word + '_synonym_list'\n",
    "#         syn_file = open(synonym_list, 'w+')\n",
    "#         similar_words = synset.lemma_names()\n",
    "#         string_similar_words = str(similar_words)\n",
    "#         syn_file.write(string_similar_words)\n",
    "#         for added_word in syn_file:\n",
    "#             added_word = added_word.strip()\n",
    "#             w1 = wordnet.synset(word)\n",
    "#             w2 = wordnet.synset(added_word) \n",
    "#             sim_level = (w1.wup_similarity(w2)) \n",
    "#             added_word_string = str(added_word)\n",
    "#             if sim_level > highest_sim_level:\n",
    "#                 highest_sim_level += sim_level\n",
    "#                 most_acc_word = added_word_string\n",
    "# print(highest_sim_level)\n",
    "# print(most_acc_word)\n",
    "\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 2,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "dict_items([('can', [Synset('can.n.01'), Synset('can.n.02'), Synset('can.n.03'), Synset('buttocks.n.01'), Synset('toilet.n.02'), Synset('toilet.n.01'), Synset('can.v.01'), Synset('displace.v.03')]), ('you', []), ('drink', [Synset('drink.n.01'), Synset('drink.n.02'), Synset('beverage.n.01'), Synset('drink.n.04'), Synset('swallow.n.02'), Synset('drink.v.01'), Synset('drink.v.02'), Synset('toast.v.02'), Synset('drink_in.v.01'), Synset('drink.v.05')]), ('alcoholic', [Synset('alcoholic.n.01'), Synset('alcoholic.a.01'), Synset('alcoholic.s.02')]), ('in', [Synset('inch.n.01'), Synset('indium.n.01'), Synset('indiana.n.01'), Synset('in.s.01'), Synset('in.s.02'), Synset('in.s.03'), Synset('in.r.01')]), ('populace', [Synset('populace.n.01')]), ('kingdom', [Synset('kingdom.n.01'), Synset('kingdom.n.02'), Synset('kingdom.n.03'), Synset('kingdom.n.04'), Synset('kingdom.n.05'), Synset('kingdom.n.06')]), ('of', []), ('denmark', [Synset('denmark.n.01')])])\n",
      "can [Synset('can.n.01'), Synset('can.n.02'), Synset('can.n.03'), Synset('buttocks.n.01'), Synset('toilet.n.02'), Synset('toilet.n.01'), Synset('can.v.01'), Synset('displace.v.03')]\n",
      "['buttocks', 'can', 'toilet', 'can ', 'displace']\n",
      "you []\n",
      "['you', 'you ']\n",
      "drink [Synset('drink.n.01'), Synset('drink.n.02'), Synset('beverage.n.01'), Synset('drink.n.04'), Synset('swallow.n.02'), Synset('drink.v.01'), Synset('drink.v.02'), Synset('toast.v.02'), Synset('drink_in.v.01'), Synset('drink.v.05')]\n",
      "['beverage', 'toast', 'drink ', 'drink_in', 'swallow', 'drink']\n",
      "alcoholic [Synset('alcoholic.n.01'), Synset('alcoholic.a.01'), Synset('alcoholic.s.02')]\n",
      "['alcoholic', 'alcoholic ']\n",
      "in [Synset('inch.n.01'), Synset('indium.n.01'), Synset('indiana.n.01'), Synset('in.s.01'), Synset('in.s.02'), Synset('in.s.03'), Synset('in.r.01')]\n",
      "['in', 'drink ', 'indium', 'inch', 'drink', 'indiana']\n",
      "populace [Synset('populace.n.01')]\n",
      "['in ', 'populace', 'in']\n",
      "kingdom [Synset('kingdom.n.01'), Synset('kingdom.n.02'), Synset('kingdom.n.03'), Synset('kingdom.n.04'), Synset('kingdom.n.05'), Synset('kingdom.n.06')]\n",
      "['populace', 'populace ', 'kingdom']\n",
      "of []\n",
      "['in ', 'in']\n",
      "denmark [Synset('denmark.n.01')]\n",
      "['kingdom ', 'denmark', 'kingdom']\n",
      "[['buttocks', 'can', 'toilet', 'can ', 'displace'], ['you', 'you '], ['beverage', 'toast', 'drink ', 'drink_in', 'swallow', 'drink'], ['alcoholic', 'alcoholic '], ['in', 'drink ', 'indium', 'inch', 'drink', 'indiana'], ['in ', 'populace', 'in'], ['populace', 'populace ', 'kingdom'], ['in ', 'in'], ['kingdom ', 'denmark', 'kingdom'], ['of', 'of '], ['denmark', 'denmark ']]\n"
     ]
    }
   ],
   "source": [
    "from itertools import chain\n",
    "from nltk.corpus import wordnet\n",
    "from nltk.tokenize import RegexpTokenizer\n",
    "tokenizer = RegexpTokenizer(\"[\\w']+\")\n",
    "synonym_sentence = 'can you drink alcoholic drink in populace in kingdom of denmark'\n",
    "components = tokenizer.tokenize(synonym_sentence)\n",
    "highest_sim_level = 0\n",
    "most_acc_word = ''\n",
    "synonyms = dict()\n",
    "split_sentence = synonym_sentence.split()\n",
    "nested_accumulator = 0\n",
    "nested_syns = [ [] for num in range(len(components)) ]\n",
    "nested_accumulator = 0\n",
    "original_accumulator = 0\n",
    "for word in components:\n",
    "    syns = wordnet.synsets(word)\n",
    "    synonyms[word] = syns\n",
    "    nested_syns[original_accumulator].append(word)\n",
    "    nested_syns[original_accumulator].append(word + ' ')\n",
    "    #nested_syns[original_accumulator].append(word)\n",
    "    original_accumulator += 1\n",
    "print(synonyms.items())\n",
    "for key, value in synonyms.items():\n",
    "    file_name = str(key) + '_syns'\n",
    "    with open(file_name, 'w') as synonym_file:\n",
    "        for syn in value:\n",
    "            synonym_file.write(str(syn) + '\\n')\n",
    "        for syn in value:\n",
    "            syn_list = str(syn)\n",
    "            syn_list = syn_list.replace(\"Synset('\", \"\")\n",
    "            cut_off = syn_list.index('.')\n",
    "            isolated_syn = str(syn_list[0:cut_off])\n",
    "            nested_syns[nested_accumulator].append(isolated_syn)\n",
    "        nested_syns[nested_accumulator] = list(set(nested_syns[nested_accumulator]))\n",
    "        print(key, value)\n",
    "        if len(nested_syns[nested_accumulator]) == 0:\n",
    "            nested_syns[nested_accumulator].append(key)\n",
    "            nested_syns[nested_accumulator].append(key + ' ')\n",
    "        elif len(nested_syns[nested_accumulator]) == 1:\n",
    "            nested_syns[nested_accumulator].append(nested_syns[nested_accumulator][0] + ' ')\n",
    "        print(nested_syns[nested_accumulator])\n",
    "        nested_accumulator += 1\n",
    "print(nested_syns)\n",
    "\n",
    "#add original word to index portion of synonyms (no empty lists)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 4,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "[['can ', 'you', 'drink_in', 'alcoholic ', 'in', 'populace', 'populace', 'in ', 'denmark', 'of', 'denmark '], ['displace', 'you ', 'drink ', 'alcoholic ', 'inch', 'in', 'populace', 'in', 'denmark', 'of ', 'denmark '], ['displace', 'you', 'toast', 'alcoholic', 'in', 'in ', 'populace ', 'in ', 'denmark', 'of ', 'denmark '], ['buttocks', 'you ', 'drink_in', 'alcoholic', 'indiana', 'in ', 'kingdom', 'in ', 'kingdom ', 'of', 'denmark'], ['toilet', 'you ', 'toast', 'alcoholic ', 'drink', 'populace', 'populace', 'in', 'denmark', 'of', 'denmark'], ['displace', 'you', 'drink ', 'alcoholic ', 'indiana', 'in ', 'populace ', 'in ', 'kingdom', 'of', 'denmark'], ['displace', 'you', 'toast', 'alcoholic', 'drink ', 'in', 'populace ', 'in ', 'denmark', 'of', 'denmark'], ['can', 'you', 'beverage', 'alcoholic ', 'indium', 'in ', 'kingdom', 'in ', 'kingdom ', 'of', 'denmark '], ['can ', 'you ', 'drink_in', 'alcoholic', 'drink', 'in', 'populace ', 'in', 'kingdom', 'of', 'denmark '], ['buttocks', 'you', 'swallow', 'alcoholic ', 'drink ', 'in', 'populace', 'in ', 'kingdom ', 'of ', 'denmark']]\n",
      "[['buttocks', 'can', 'toilet', 'can ', 'displace'], ['you', 'you '], ['beverage', 'toast', 'drink ', 'drink_in', 'swallow', 'drink'], ['alcoholic', 'alcoholic '], ['in', 'drink ', 'indium', 'inch', 'drink', 'indiana'], ['in ', 'populace', 'in'], ['populace', 'populace ', 'kingdom'], ['in ', 'in'], ['kingdom ', 'denmark', 'kingdom'], ['of', 'of '], ['denmark', 'denmark ']]\n",
      "[5, 2, 6, 2, 6, 3, 3, 2, 3, 2, 2]\n",
      "40\n",
      "80\n",
      "160\n",
      "[['can ', 'you', 'swallow', 'alcoholic ', 'in', 'populace', 'kingdom', 'in ', 'kingdom', 'of', 'denmark '], ['buttocks', 'you ', 'drink_in', 'alcoholic', 'inch', 'in', 'populace ', 'in', 'kingdom', 'of ', 'denmark '], ['displace', 'you', 'swallow', 'alcoholic ', 'inch', 'populace', 'populace', 'in ', 'denmark', 'of', 'denmark'], ['toilet', 'you ', 'drink', 'alcoholic', 'drink', 'populace', 'populace ', 'in', 'kingdom', 'of ', 'denmark'], ['toilet', 'you', 'drink', 'alcoholic ', 'indiana', 'in ', 'populace ', 'in ', 'kingdom ', 'of', 'denmark'], ['can', 'you', 'toast', 'alcoholic', 'drink ', 'in', 'kingdom', 'in ', 'denmark', 'of', 'denmark '], ['can ', 'you ', 'swallow', 'alcoholic', 'indiana', 'in', 'populace', 'in', 'kingdom', 'of', 'denmark'], ['can ', 'you ', 'drink_in', 'alcoholic', 'indium', 'in', 'kingdom', 'in ', 'denmark', 'of ', 'denmark '], ['can ', 'you ', 'drink_in', 'alcoholic ', 'drink ', 'populace', 'kingdom', 'in', 'kingdom ', 'of ', 'denmark '], ['buttocks', 'you', 'drink ', 'alcoholic ', 'in', 'in', 'populace ', 'in', 'kingdom ', 'of', 'denmark '], ['can ', 'you ', 'toast', 'alcoholic', 'in', 'in', 'populace', 'in', 'kingdom', 'of', 'denmark '], ['can ', 'you', 'drink_in', 'alcoholic ', 'drink', 'in', 'kingdom', 'in', 'denmark', 'of ', 'denmark'], ['can ', 'you ', 'drink', 'alcoholic', 'drink', 'in', 'populace ', 'in ', 'denmark', 'of ', 'denmark '], ['buttocks', 'you', 'toast', 'alcoholic ', 'in', 'in ', 'populace ', 'in ', 'denmark', 'of ', 'denmark '], ['toilet', 'you ', 'drink ', 'alcoholic ', 'indiana', 'populace', 'populace', 'in', 'denmark', 'of', 'denmark'], ['buttocks', 'you ', 'drink_in', 'alcoholic ', 'indiana', 'in ', 'kingdom', 'in', 'kingdom ', 'of ', 'denmark'], ['can', 'you ', 'drink ', 'alcoholic ', 'inch', 'in', 'kingdom', 'in', 'kingdom', 'of ', 'denmark'], ['buttocks', 'you ', 'toast', 'alcoholic ', 'in', 'populace', 'populace', 'in', 'kingdom ', 'of', 'denmark'], ['toilet', 'you', 'drink ', 'alcoholic ', 'drink', 'in', 'populace ', 'in ', 'kingdom', 'of', 'denmark '], ['displace', 'you', 'swallow', 'alcoholic ', 'inch', 'populace', 'populace', 'in', 'denmark', 'of', 'denmark '], ['can ', 'you', 'drink', 'alcoholic', 'indiana', 'in ', 'populace ', 'in ', 'denmark', 'of ', 'denmark'], ['can', 'you', 'drink ', 'alcoholic ', 'indium', 'populace', 'kingdom', 'in', 'kingdom', 'of', 'denmark'], ['buttocks', 'you', 'toast', 'alcoholic', 'indiana', 'populace', 'populace', 'in ', 'denmark', 'of', 'denmark'], ['can', 'you', 'drink ', 'alcoholic', 'indium', 'populace', 'populace ', 'in ', 'denmark', 'of', 'denmark'], ['buttocks', 'you ', 'swallow', 'alcoholic', 'drink ', 'in', 'populace', 'in', 'denmark', 'of ', 'denmark'], ['can ', 'you ', 'swallow', 'alcoholic', 'indium', 'populace', 'populace', 'in ', 'denmark', 'of ', 'denmark'], ['toilet', 'you', 'drink ', 'alcoholic', 'inch', 'populace', 'populace', 'in ', 'denmark', 'of', 'denmark '], ['can', 'you ', 'swallow', 'alcoholic', 'in', 'in ', 'kingdom', 'in ', 'denmark', 'of', 'denmark'], ['can', 'you', 'toast', 'alcoholic', 'in', 'populace', 'kingdom', 'in ', 'kingdom ', 'of ', 'denmark '], ['toilet', 'you ', 'drink_in', 'alcoholic', 'drink', 'in', 'populace', 'in ', 'denmark', 'of', 'denmark '], ['buttocks', 'you ', 'drink_in', 'alcoholic', 'indium', 'in ', 'populace ', 'in', 'kingdom ', 'of ', 'denmark'], ['buttocks', 'you', 'drink ', 'alcoholic', 'indium', 'populace', 'populace', 'in ', 'kingdom ', 'of', 'denmark'], ['can ', 'you ', 'drink', 'alcoholic', 'inch', 'in ', 'populace', 'in ', 'denmark', 'of', 'denmark'], ['toilet', 'you', 'drink', 'alcoholic', 'inch', 'in ', 'populace ', 'in ', 'denmark', 'of ', 'denmark '], ['buttocks', 'you', 'drink', 'alcoholic', 'drink', 'in ', 'populace ', 'in ', 'kingdom', 'of ', 'denmark '], ['toilet', 'you ', 'beverage', 'alcoholic ', 'drink ', 'populace', 'kingdom', 'in', 'kingdom', 'of ', 'denmark'], ['buttocks', 'you ', 'drink', 'alcoholic', 'in', 'in', 'kingdom', 'in', 'denmark', 'of ', 'denmark '], ['toilet', 'you ', 'drink ', 'alcoholic', 'in', 'in ', 'populace', 'in', 'denmark', 'of', 'denmark'], ['can', 'you ', 'drink ', 'alcoholic ', 'drink ', 'in', 'populace', 'in ', 'kingdom', 'of', 'denmark'], ['toilet', 'you ', 'drink', 'alcoholic', 'indium', 'in', 'populace ', 'in ', 'kingdom', 'of ', 'denmark'], ['toilet', 'you', 'swallow', 'alcoholic', 'indium', 'in', 'populace', 'in', 'kingdom', 'of', 'denmark '], ['can', 'you ', 'swallow', 'alcoholic', 'in', 'populace', 'populace', 'in ', 'kingdom', 'of ', 'denmark '], ['can', 'you ', 'beverage', 'alcoholic ', 'in', 'in', 'populace ', 'in', 'kingdom', 'of ', 'denmark'], ['displace', 'you', 'toast', 'alcoholic ', 'in', 'populace', 'populace ', 'in', 'denmark', 'of ', 'denmark '], ['displace', 'you ', 'drink ', 'alcoholic ', 'drink ', 'in ', 'kingdom', 'in ', 'denmark', 'of ', 'denmark'], ['buttocks', 'you', 'drink', 'alcoholic ', 'indium', 'populace', 'kingdom', 'in ', 'kingdom ', 'of', 'denmark '], ['can ', 'you ', 'beverage', 'alcoholic ', 'inch', 'populace', 'populace ', 'in ', 'kingdom ', 'of', 'denmark'], ['buttocks', 'you ', 'drink_in', 'alcoholic ', 'indium', 'in', 'populace', 'in ', 'kingdom ', 'of ', 'denmark'], ['toilet', 'you ', 'drink_in', 'alcoholic ', 'indiana', 'in', 'populace', 'in ', 'kingdom', 'of', 'denmark '], ['toilet', 'you ', 'drink ', 'alcoholic', 'indium', 'populace', 'kingdom', 'in ', 'denmark', 'of ', 'denmark '], ['toilet', 'you', 'swallow', 'alcoholic ', 'in', 'in', 'populace ', 'in ', 'kingdom ', 'of ', 'denmark '], ['toilet', 'you ', 'swallow', 'alcoholic ', 'drink ', 'in ', 'populace', 'in ', 'kingdom', 'of ', 'denmark '], ['displace', 'you ', 'toast', 'alcoholic ', 'drink ', 'in', 'populace', 'in', 'kingdom ', 'of', 'denmark '], ['can ', 'you ', 'drink_in', 'alcoholic', 'drink ', 'populace', 'populace ', 'in ', 'denmark', 'of', 'denmark '], ['toilet', 'you ', 'toast', 'alcoholic', 'drink ', 'populace', 'kingdom', 'in', 'denmark', 'of', 'denmark '], ['can', 'you', 'beverage', 'alcoholic ', 'inch', 'in ', 'kingdom', 'in ', 'kingdom ', 'of', 'denmark'], ['displace', 'you', 'swallow', 'alcoholic ', 'inch', 'in ', 'kingdom', 'in', 'kingdom ', 'of ', 'denmark '], ['buttocks', 'you ', 'drink_in', 'alcoholic ', 'in', 'in ', 'populace', 'in', 'denmark', 'of', 'denmark'], ['buttocks', 'you', 'drink_in', 'alcoholic ', 'indiana', 'in ', 'kingdom', 'in ', 'kingdom ', 'of', 'denmark'], ['toilet', 'you', 'toast', 'alcoholic ', 'inch', 'in', 'populace ', 'in ', 'kingdom', 'of', 'denmark'], ['displace', 'you ', 'toast', 'alcoholic ', 'in', 'populace', 'populace', 'in ', 'kingdom ', 'of', 'denmark '], ['can ', 'you', 'toast', 'alcoholic', 'drink ', 'populace', 'populace ', 'in ', 'kingdom ', 'of', 'denmark '], ['can ', 'you', 'drink', 'alcoholic ', 'in', 'in', 'kingdom', 'in', 'denmark', 'of ', 'denmark'], ['can ', 'you ', 'drink ', 'alcoholic', 'in', 'in', 'populace ', 'in ', 'kingdom ', 'of', 'denmark '], ['displace', 'you', 'drink', 'alcoholic ', 'indiana', 'populace', 'populace ', 'in ', 'denmark', 'of', 'denmark '], ['can ', 'you ', 'drink_in', 'alcoholic', 'in', 'populace', 'kingdom', 'in', 'denmark', 'of', 'denmark '], ['displace', 'you', 'drink ', 'alcoholic ', 'drink ', 'in', 'kingdom', 'in', 'kingdom ', 'of ', 'denmark'], ['can ', 'you', 'toast', 'alcoholic', 'indiana', 'populace', 'populace', 'in ', 'denmark', 'of', 'denmark'], ['displace', 'you', 'beverage', 'alcoholic', 'indium', 'in', 'kingdom', 'in', 'denmark', 'of ', 'denmark'], ['can', 'you', 'drink ', 'alcoholic ', 'drink ', 'populace', 'kingdom', 'in ', 'kingdom ', 'of ', 'denmark '], ['can', 'you ', 'beverage', 'alcoholic', 'indium', 'in ', 'kingdom', 'in', 'denmark', 'of', 'denmark '], ['can', 'you', 'toast', 'alcoholic', 'in', 'in ', 'populace', 'in ', 'denmark', 'of ', 'denmark '], ['can', 'you ', 'toast', 'alcoholic ', 'inch', 'in ', 'populace', 'in', 'kingdom', 'of', 'denmark'], ['toilet', 'you', 'drink_in', 'alcoholic ', 'drink', 'in ', 'populace', 'in', 'kingdom', 'of ', 'denmark'], ['buttocks', 'you', 'swallow', 'alcoholic', 'in', 'in ', 'populace', 'in', 'kingdom', 'of ', 'denmark'], ['can', 'you ', 'swallow', 'alcoholic', 'inch', 'populace', 'kingdom', 'in ', 'kingdom ', 'of ', 'denmark'], ['can ', 'you', 'drink ', 'alcoholic ', 'inch', 'populace', 'populace', 'in ', 'denmark', 'of ', 'denmark '], ['can', 'you', 'beverage', 'alcoholic ', 'drink', 'populace', 'populace', 'in ', 'kingdom ', 'of', 'denmark '], ['can ', 'you', 'drink ', 'alcoholic ', 'indium', 'in', 'kingdom', 'in', 'kingdom ', 'of ', 'denmark '], ['displace', 'you', 'beverage', 'alcoholic', 'indiana', 'in ', 'populace', 'in ', 'kingdom', 'of', 'denmark '], ['displace', 'you ', 'swallow', 'alcoholic', 'indium', 'in', 'populace', 'in', 'kingdom ', 'of ', 'denmark'], ['displace', 'you ', 'drink', 'alcoholic ', 'inch', 'in ', 'populace', 'in', 'kingdom', 'of', 'denmark'], ['buttocks', 'you', 'swallow', 'alcoholic ', 'indium', 'in', 'kingdom', 'in', 'kingdom', 'of', 'denmark '], ['buttocks', 'you', 'toast', 'alcoholic ', 'drink', 'in', 'populace ', 'in', 'kingdom', 'of ', 'denmark '], ['can', 'you ', 'toast', 'alcoholic ', 'indiana', 'in', 'populace', 'in', 'kingdom ', 'of ', 'denmark '], ['toilet', 'you', 'drink ', 'alcoholic ', 'drink', 'populace', 'kingdom', 'in', 'denmark', 'of ', 'denmark '], ['toilet', 'you', 'drink', 'alcoholic', 'indiana', 'in ', 'kingdom', 'in', 'denmark', 'of', 'denmark'], ['can', 'you', 'drink', 'alcoholic ', 'indium', 'in', 'populace', 'in', 'kingdom', 'of ', 'denmark'], ['displace', 'you ', 'toast', 'alcoholic ', 'drink ', 'in', 'populace ', 'in ', 'kingdom', 'of ', 'denmark '], ['can', 'you', 'swallow', 'alcoholic', 'indiana', 'in ', 'kingdom', 'in', 'kingdom', 'of ', 'denmark '], ['buttocks', 'you ', 'drink ', 'alcoholic ', 'inch', 'in ', 'kingdom', 'in ', 'kingdom ', 'of ', 'denmark'], ['toilet', 'you ', 'drink ', 'alcoholic ', 'indium', 'in', 'populace', 'in', 'denmark', 'of', 'denmark '], ['toilet', 'you ', 'toast', 'alcoholic', 'indium', 'in', 'populace ', 'in ', 'kingdom', 'of ', 'denmark'], ['buttocks', 'you ', 'drink_in', 'alcoholic', 'indium', 'in ', 'populace ', 'in ', 'denmark', 'of ', 'denmark '], ['displace', 'you', 'beverage', 'alcoholic', 'drink', 'in ', 'kingdom', 'in ', 'kingdom', 'of', 'denmark'], ['can ', 'you ', 'drink_in', 'alcoholic', 'in', 'in', 'populace ', 'in', 'kingdom ', 'of ', 'denmark'], ['displace', 'you ', 'drink_in', 'alcoholic ', 'inch', 'in ', 'populace', 'in', 'denmark', 'of ', 'denmark'], ['can ', 'you ', 'beverage', 'alcoholic', 'in', 'in ', 'kingdom', 'in', 'kingdom ', 'of', 'denmark '], ['displace', 'you ', 'drink', 'alcoholic ', 'in', 'populace', 'populace', 'in', 'kingdom', 'of', 'denmark '], ['buttocks', 'you ', 'toast', 'alcoholic', 'drink', 'in', 'populace', 'in', 'kingdom ', 'of ', 'denmark '], ['toilet', 'you', 'toast', 'alcoholic', 'indiana', 'populace', 'populace ', 'in ', 'kingdom ', 'of', 'denmark'], ['can', 'you', 'toast', 'alcoholic ', 'drink', 'in', 'kingdom', 'in ', 'denmark', 'of ', 'denmark'], ['can', 'you ', 'drink', 'alcoholic', 'drink ', 'in', 'kingdom', 'in', 'denmark', 'of', 'denmark'], ['displace', 'you ', 'swallow', 'alcoholic ', 'drink', 'in', 'populace ', 'in', 'kingdom', 'of', 'denmark '], ['toilet', 'you ', 'drink_in', 'alcoholic ', 'inch', 'populace', 'populace', 'in ', 'denmark', 'of ', 'denmark'], ['displace', 'you ', 'drink ', 'alcoholic ', 'in', 'in ', 'populace', 'in ', 'kingdom', 'of ', 'denmark '], ['can', 'you', 'swallow', 'alcoholic ', 'inch', 'in', 'populace ', 'in ', 'kingdom ', 'of ', 'denmark '], ['toilet', 'you ', 'drink_in', 'alcoholic ', 'drink ', 'populace', 'populace', 'in', 'denmark', 'of ', 'denmark'], ['can ', 'you ', 'drink_in', 'alcoholic', 'inch', 'populace', 'populace ', 'in', 'kingdom', 'of', 'denmark '], ['can', 'you', 'drink_in', 'alcoholic ', 'drink', 'in', 'populace', 'in ', 'kingdom ', 'of', 'denmark'], ['can ', 'you ', 'swallow', 'alcoholic', 'in', 'populace', 'populace', 'in', 'denmark', 'of', 'denmark'], ['buttocks', 'you ', 'drink', 'alcoholic ', 'drink', 'in', 'kingdom', 'in', 'kingdom ', 'of ', 'denmark'], ['toilet', 'you ', 'beverage', 'alcoholic', 'inch', 'populace', 'kingdom', 'in ', 'denmark', 'of', 'denmark'], ['can ', 'you', 'swallow', 'alcoholic', 'in', 'in ', 'kingdom', 'in ', 'denmark', 'of ', 'denmark'], ['can ', 'you ', 'beverage', 'alcoholic', 'inch', 'populace', 'populace', 'in ', 'kingdom', 'of ', 'denmark '], ['toilet', 'you', 'toast', 'alcoholic ', 'drink ', 'in', 'populace', 'in ', 'denmark', 'of', 'denmark '], ['can', 'you', 'toast', 'alcoholic ', 'indium', 'populace', 'populace ', 'in ', 'kingdom', 'of', 'denmark '], ['toilet', 'you ', 'toast', 'alcoholic ', 'drink ', 'populace', 'kingdom', 'in ', 'denmark', 'of', 'denmark'], ['can', 'you', 'swallow', 'alcoholic ', 'inch', 'in ', 'kingdom', 'in', 'kingdom ', 'of ', 'denmark'], ['can', 'you ', 'swallow', 'alcoholic ', 'indiana', 'in ', 'kingdom', 'in ', 'kingdom', 'of ', 'denmark '], ['toilet', 'you ', 'drink_in', 'alcoholic ', 'inch', 'in ', 'populace', 'in ', 'kingdom ', 'of ', 'denmark '], ['can', 'you', 'beverage', 'alcoholic ', 'in', 'in', 'kingdom', 'in ', 'kingdom ', 'of', 'denmark'], ['can ', 'you ', 'beverage', 'alcoholic ', 'indiana', 'in ', 'populace ', 'in', 'denmark', 'of ', 'denmark '], ['can ', 'you', 'swallow', 'alcoholic ', 'in', 'in ', 'populace', 'in', 'kingdom ', 'of', 'denmark'], ['displace', 'you ', 'beverage', 'alcoholic ', 'indiana', 'populace', 'kingdom', 'in', 'kingdom', 'of', 'denmark'], ['can', 'you', 'toast', 'alcoholic ', 'in', 'populace', 'kingdom', 'in ', 'denmark', 'of', 'denmark'], ['displace', 'you ', 'drink_in', 'alcoholic', 'indium', 'populace', 'kingdom', 'in ', 'denmark', 'of', 'denmark '], ['buttocks', 'you ', 'swallow', 'alcoholic ', 'inch', 'in ', 'populace', 'in ', 'denmark', 'of', 'denmark '], ['buttocks', 'you', 'toast', 'alcoholic ', 'inch', 'populace', 'populace ', 'in ', 'kingdom ', 'of', 'denmark '], ['displace', 'you ', 'drink ', 'alcoholic', 'drink ', 'in', 'kingdom', 'in', 'denmark', 'of ', 'denmark'], ['toilet', 'you', 'drink ', 'alcoholic', 'drink ', 'populace', 'populace ', 'in ', 'denmark', 'of', 'denmark'], ['can ', 'you', 'drink ', 'alcoholic', 'drink ', 'in ', 'kingdom', 'in', 'kingdom ', 'of', 'denmark '], ['can ', 'you ', 'drink', 'alcoholic', 'inch', 'in', 'populace ', 'in ', 'denmark', 'of', 'denmark '], ['displace', 'you', 'drink_in', 'alcoholic ', 'indium', 'in', 'populace ', 'in', 'denmark', 'of', 'denmark'], ['buttocks', 'you', 'beverage', 'alcoholic ', 'inch', 'in', 'kingdom', 'in ', 'denmark', 'of ', 'denmark '], ['can ', 'you', 'drink_in', 'alcoholic ', 'drink', 'in ', 'kingdom', 'in', 'denmark', 'of ', 'denmark '], ['buttocks', 'you', 'toast', 'alcoholic', 'in', 'in ', 'kingdom', 'in', 'kingdom ', 'of ', 'denmark'], ['can ', 'you', 'swallow', 'alcoholic', 'drink ', 'in', 'populace ', 'in', 'kingdom ', 'of', 'denmark'], ['toilet', 'you ', 'drink ', 'alcoholic ', 'inch', 'in', 'populace', 'in ', 'kingdom', 'of ', 'denmark '], ['can ', 'you ', 'drink ', 'alcoholic ', 'drink ', 'populace', 'kingdom', 'in ', 'kingdom', 'of', 'denmark'], ['can', 'you ', 'swallow', 'alcoholic ', 'drink ', 'in', 'kingdom', 'in ', 'denmark', 'of', 'denmark '], ['can', 'you', 'drink ', 'alcoholic', 'indium', 'in', 'kingdom', 'in ', 'denmark', 'of', 'denmark '], ['toilet', 'you', 'toast', 'alcoholic', 'inch', 'in ', 'populace ', 'in ', 'kingdom', 'of', 'denmark '], ['displace', 'you', 'beverage', 'alcoholic ', 'inch', 'in', 'kingdom', 'in', 'kingdom ', 'of ', 'denmark'], ['toilet', 'you', 'drink', 'alcoholic', 'in', 'in', 'populace ', 'in', 'denmark', 'of', 'denmark'], ['can ', 'you', 'toast', 'alcoholic', 'drink', 'in ', 'populace ', 'in', 'kingdom ', 'of ', 'denmark'], ['can', 'you ', 'drink_in', 'alcoholic', 'drink', 'in ', 'populace', 'in ', 'denmark', 'of', 'denmark'], ['buttocks', 'you', 'drink_in', 'alcoholic ', 'drink ', 'populace', 'kingdom', 'in ', 'kingdom', 'of', 'denmark'], ['can', 'you', 'toast', 'alcoholic ', 'inch', 'populace', 'kingdom', 'in', 'denmark', 'of ', 'denmark '], ['toilet', 'you ', 'drink ', 'alcoholic ', 'indiana', 'populace', 'kingdom', 'in', 'kingdom', 'of', 'denmark '], ['can', 'you', 'beverage', 'alcoholic ', 'drink ', 'populace', 'populace ', 'in', 'kingdom ', 'of ', 'denmark'], ['buttocks', 'you', 'toast', 'alcoholic', 'drink ', 'in ', 'kingdom', 'in', 'kingdom', 'of ', 'denmark'], ['can ', 'you', 'toast', 'alcoholic', 'drink', 'populace', 'populace ', 'in ', 'kingdom ', 'of ', 'denmark'], ['can ', 'you ', 'swallow', 'alcoholic ', 'drink', 'in', 'populace ', 'in ', 'kingdom ', 'of ', 'denmark'], ['can ', 'you', 'drink ', 'alcoholic ', 'inch', 'in ', 'populace ', 'in ', 'kingdom', 'of', 'denmark'], ['can ', 'you', 'drink ', 'alcoholic ', 'in', 'in ', 'populace ', 'in', 'kingdom', 'of', 'denmark'], ['can', 'you ', 'swallow', 'alcoholic ', 'drink', 'in', 'populace', 'in', 'kingdom ', 'of ', 'denmark'], ['can ', 'you', 'toast', 'alcoholic ', 'inch', 'populace', 'kingdom', 'in ', 'denmark', 'of ', 'denmark '], ['can ', 'you ', 'drink_in', 'alcoholic ', 'inch', 'in', 'populace', 'in', 'kingdom', 'of', 'denmark '], ['buttocks', 'you ', 'drink', 'alcoholic ', 'in', 'in ', 'kingdom', 'in ', 'kingdom', 'of ', 'denmark']]\n"
     ]
    }
   ],
   "source": [
    "import random\n",
    "#from Gregory_CoveringArray import verify_covering_array\n",
    "from Gregory_LocatingArray import locating_array_verifier\n",
    "import pprint\n",
    "\n",
    "t = 2\n",
    "k = len(nested_syns)\n",
    "N = 10\n",
    "d = 1\n",
    "\n",
    "def sentence_covering_array(values, N):\n",
    "    ca_list = []\n",
    "    for _ in range(N):\n",
    "        new_rows = list(map(random.choice, values))\n",
    "        ca_list.append(new_rows)\n",
    "    return ca_list\n",
    "\n",
    "print(sentence_covering_array(nested_syns, N))\n",
    "\n",
    "if locating_array_verifier(sentence_covering_array(nested_syns, N), t, list(map(len, nested_syns)), d, nested_syns):\n",
    "    print(f'This is a locating array!')\n",
    "else:\n",
    "    N *= 2\n",
    "    print(nested_syns)\n",
    "    print(list(map(len, nested_syns)))\n",
    "    test = locating_array_verifier(sentence_covering_array(nested_syns, N), t, list(map(len, nested_syns)), d, nested_syns)\n",
    "    while test != True:\n",
    "            N *= 2\n",
    "            var = sentence_covering_array(nested_syns, N)\n",
    "            test = locating_array_verifier(var, t, list(map(len, nested_syns)), d, nested_syns)\n",
    "            print(N)\n",
    "print(var)\n",
    "            "
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.11.4"
  },
  "orig_nbformat": 4
 },
 "nbformat": 4,
 "nbformat_minor": 2
}


{'cells': [{'cell_type': 'code',
   'execution_count': 9,
   'metadata': {},
   'outputs': [{'name': 'stdout',
     'output_type': 'stream',
     'text': ["['ace', 'adept', 'champion', 'sensation', 'maven', 'mavin', 'virtuoso', 'genius', 'hotshot', 'star', 'superstar', 'whiz', 'whizz', 'wizard', 'wiz']\n",
      "['whiz']\n",
      "['whizz', 'whiz', 'whirr', 'whir', 'birr', 'purr']\n"]}],
   'source': ['from itertools import chain\n',
    'from nltk.corpus import wordnet\n',
    "# for synset in wordnet.synsets('whiz'): \n",
    '#     print(synset.lemma_names())']},
  {'cell_type': 'code',
   'execution_count': 8,
   'metadata': {},
   'outputs': [{'name': 'stderr',
     'output_type': 'stream',
     'text': ['[nltk_data] Downloading package wordnet to\n',
      '[nltk_data]     C:\\Users\\william.gregory\\AppData\\Roaming\\nltk_data...\n']},
    {'data': {'text/plain': ['True']},
     'execution_count': 8,
     'metadata': {},
     'output_type': 'execute_result'}],
   'source': ['i